## Implementation of Neural style transfer
References:
- https://arxiv.org/pdf/1508.06576.pdf
- https://keras.io/examples/neural_style_transfer/

In [31]:
# Set paths to your files
content_image_path = './content_image.jpg'
style_image_path = './style_image.jpg'
output_image_path = './generated_image.jpg'

# which VGG19 layers to user for style cost
style_layers_names = [
    ('block1_conv1'),
    ('block2_conv1'),
    ('block3_conv1'),
    ('block4_conv1'),
    ('block5_conv1')]

# component weights in loss function
total_variation_weight = 2e2
style_weight = 1e2
content_weight = 7.5e0

# num of iterations for optimization
iters = 20

# number of rows for the generated picture
img_nrows = 300

In [3]:
from keras.applications import VGG19
from keras.preprocessing.image import load_img, save_img
from keras.preprocessing.image import img_to_array, array_to_img
from keras.applications.vgg19 import preprocess_input
from keras import backend as K
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from scipy.optimize import fmin_l_bfgs_b
import matplotlib.pyplot as plt
import imageio

Using TensorFlow backend.


In [4]:
def noise_image(content_image, img_nrows, img_ncols, noise_ratio=.3):
    """
    Given a content image tensor add random noise and return image
    """
    noise_image = np.random.uniform(-20, 20, (1, img_nrows, img_ncols, 3)).astype('float32')
    input_image = noise_image * noise_image * noise_ratio + content_image * (1 - noise_ratio)
    return input_image

In [5]:
def compute_content_cost(content_activation, generated_content_activation):
    """
    Given the content activation and generated picture activation of the same conv layer
    return cost over all elements
    """
    return K.sum(K.square(generated_content_activation - content_activation))

In [6]:
def gram_matrix(A):
    """
    Given a tensor return the gram matrix
    """
    features = K.batch_flatten(K.permute_dimensions(A, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [10]:
def compute_layer_style_cost(style_activation, generated_activation):
    """
    Given the style activation and generated picture activation of the same conv layer
    return the loss as distance of the gram matrices of both activations
    """
    W, H, C = img_ncols, img_nrows, 3
    GS = gram_matrix(style_activation)
    GG = gram_matrix(generated_activation)
    factor = 1. / float(2 * C * H * W) ** 2
    return factor * K.sum(K.square(GG - GS))

In [8]:
def compute_style_cost(style_layers):
    """
    Given the activation of multiple conv layers
    return the weighted sum of the individual style layer cost
    """
    # intialize loss
    J_style = 0
    # all layers are equally weighted (simple)
    layer_weight = 1. / len(style_layers)

    # loop over style layers to increment loss
    for layer in style_layers:
        style_activation = layer[1, :, :, :]
        generated_activation = layer[2, :, :, :]

        # Compute style_cost for the current layer
        J_style += layer_weight * compute_layer_style_cost(style_activation,
                                                           generated_activation)
    return J_style

In [33]:
def variation_loss(generated_image):
    """
    return variation loss given current image generated, smoothes image
    """
    a = K.square(generated_image[:, :img_nrows-1, :img_ncols-1,
                                 :] - generated_image[:, 1:, :img_ncols-1, :])
    b = K.square(generated_image[:, :img_nrows-1, :img_ncols-1,
                                 :] - generated_image[:, :img_nrows-1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [24]:
def total_cost(content_layers, 
               style_layers, 
               generated_image, 
               total_variation_weight, 
               style_weight, 
               content_weight):
    """
    given the activation of content layer and style layers for generated and input pics
    return weighted total loss
    """
    
    # 1. Content cost - grab the correct activations from the content tensor
    content_activation = content_layers[0, :, :, :]
    generated_content_activation = content_layers[2, :, :, :]
    J_content = compute_content_cost(content_activation, generated_content_activation)
    
    # 2. Style cost - compute the syle cost over all style layers
    J_style = compute_style_cost(style_layers)
    
    # 3. Total variation loss - compute the total variation loss
    J_vloss = variation_loss(generated_image)
    
    return content_weight * J_content + style_weight * J_style + total_variation_weight * J_vloss

In [25]:
def get_layers(model, style_layers_names):
    """
    given (VGG19) mode, return the desired style and content layers
    """ 
    layers = dict([(layer.name, layer.output) for layer in model.layers])
    content_layers = layers['block4_conv2'] # hard coded content layer
    style_layers = [layers[layer] for layer in style_layers_names]
    return content_layers, style_layers

In [26]:
class Evaluator(object):
    """
    Evaluator class used to track gradients and loss values together
    Reference: https://keras.io/examples/neural_style_transfer/ 
    """

    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        
        generated = x
        generated = generated.reshape((1, img_nrows, img_ncols, 3))
        outs = f_outputs([generated])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [27]:
def deprocess_image(x):
    """
    Remove zero center and reorder to RGB
    Reference: https://keras.io/examples/neural_style_transfer/ 
    """
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [28]:
def preprocess_image(path, img_nrows, img_ncols):
    """
    given path, load image, scale, exand and perform VGG19 preprocessing
    """
    img = load_img(path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [ ]:
# get dimensions for output picture
width, height = load_img(content_image_path).size
img_ncols = int(width * img_nrows / height)

# preprocess content and style picture
content_image = preprocess_image(content_image_path, img_nrows, img_ncols)
style_image = preprocess_image(style_image_path, img_nrows, img_ncols)

# generate Keras variables for pictures
content_image = K.variable(content_image)
style_image = K.variable(style_image)
generated_image = K.placeholder((1, img_nrows, img_ncols, 3))

# initalize cost variable
cost = K.variable(0)

# input tensor will concist of c, s and g pics
input_tensor = K.concatenate(
    [content_image, style_image, generated_image], axis=0)

# import VGG19 model
model = VGG19(input_tensor=input_tensor,
              weights='imagenet', include_top=False)

# get layers on which content and styles loss is computed
content_layers, style_layers = get_layers(model, style_layers_names)

# initialize cost
cost = total_cost(content_layers, style_layers,
                  generated_image, total_variation_weight, style_weight, content_weight)

# Initialize gradients, Keras function and Evaluator
grads = K.gradients(cost, generated_image)[0]
f_outputs = K.function([generated_image], [cost, grads])
evaluator = Evaluator()

# currently generated image starting point is content image, but can add noise
generated_img = preprocess_image(content_image_path, img_nrows, img_ncols)

for i in tqdm(range(iters)):
    generated_img, min_val, info = fmin_l_bfgs_b(evaluator.loss, generated_img,
                                                 fprime=evaluator.grads, maxfun=20)
    img = deprocess_image(generated_img.copy())
    imageio.imwrite(output_image_path, img)